In [1]:
from dotenv import load_dotenv
import os

load_dotenv()

hf_token= os.getenv("HF_TOKEN")

### Serverless API

In the Hugging Face ecosystem, there is convenient feature called Serverless API that allows you to easily run inference on many models. There's no installation or deployment required.

In [2]:
from huggingface_hub import InferenceClient

os.environ["HF_TOKEN"]= hf_token

client = InferenceClient("meta-llama/Llama-3.2-3B-Instruct")

In [3]:
client

<InferenceClient(model='meta-llama/Llama-3.2-3B-Instruct', timeout=None)>

## Tokenizer

In [4]:
from transformers import AutoTokenizer

In [5]:
tokenizer= AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-3B-Instruct")

## Agent Building

In [ ]:
pip install smolagents

In [6]:
from smolagents import CodeAgent, DuckDuckGoSearchTool, HfApiModel, load_tool, tool
import datetime
import requests
import pytz
import yaml
from tools.final_answer import FinalAnswerTool

from Gradio_UI import GradioUI

ModuleNotFoundError: No module named 'smolagents'

In [ ]:
# Below is an example of a tool that does nothing. Amaze us with your creativity!
@tool
def my_custom_tool(arg1:str, arg2:int)-> str: # it's important to specify the return type
    # Keep this format for the tool description / args description but feel free to modify the tool
    """A tool that does nothing yet 
    Args:
        arg1: the first argument
        arg2: the second argument
    """
    return "What magic will you build ?"

@tool
def get_current_time_in_timezone(timezone: str) -> str:
    """A tool that fetches the current local time in a specified timezone.
    Args:
        timezone: A string representing a valid timezone (e.g., 'America/New_York').
    """
    try:
        # Create timezone object
        tz = pytz.timezone(timezone)
        # Get current time in that timezone
        local_time = datetime.datetime.now(tz).strftime("%Y-%m-%d %H:%M:%S")
        return f"The current local time in {timezone} is: {local_time}"
    except Exception as e:
        return f"Error fetching time for timezone '{timezone}': {str(e)}"


In [ ]:
final_answer = FinalAnswerTool()
    model = HfApiModel(
    max_tokens=2096,
    temperature=0.5,
    model_id='Qwen/Qwen2.5-Coder-32B-Instruct',
    custom_role_conversions=None,
)


# Import tool from Hub
image_generation_tool = load_tool("agents-course/text-to-image", trust_remote_code=True)

# Load system prompt from prompt.yaml file
with open("prompts.yaml", 'r') as stream:
    prompt_templates = yaml.safe_load(stream)
    
agent = CodeAgent(
    model=model,
    tools=[final_answer], # add your tools here (don't remove final_answer)
    max_steps=6,
    verbosity_level=1,
    grammar=None,
    planning_interval=None,
    name=None,
    description=None,
    prompt_templates=prompt_templates # Pass system prompt to CodeAgent
)


GradioUI(agent).launch()